This script is for creating figures for PowerPoint slides:
dFFdLL - delta F by F devided by delta L by L
Delay - time difference between starting points in Calcium transients and Sarcomere Shortening
A/B - amplitude of Calcium transients devided by Calcium transients baseline
Transients_F - Calcium transient with SNR and residuals
Transients_L - Sarcomere shortening with SNR and residuals

In [3]:
#dFFdLL linear regression EXAMPLE
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import numpy as np

# Set the path to the directory containing the files
directory_path = r'D:\My dissertation\obesity\23-08-21\Animal1.mdb'


# Loop over the directories, subdirectories, and files in the directory path
for root, dirs, files in os.walk(directory_path):
    # Loop over the files in the directory
    for filename in files:
                    
        # Check if the file is a .xlsx file containing the substring "_F_analysis.xlsx"
        if not filename.endswith('.xlsx') or '_F_analysis.xlsx' not in filename:
            continue
        animal_folder = os.path.basename(os.path.dirname(root))
        # Print the current directory path
        print(root)
        # Find the keyword (Running or Sedentary) in the root path
        root_parts = os.path.normpath(root).split(os.sep)
        keyword = ""
        for part in root_parts:
            if "Running" in part or "Sedentary" in part:
                keyword = part
                break
        
        # Extract the folder name from the root path
        folder_name = os.path.basename(root)
       
        # Read in the data
        file_path = os.path.join(root, filename)
        
        # Try to get the matching L file for the current F file
        l_file_path = file_path.replace('_F_analysis.xlsx', '_L_analysis.xlsx')
        if not os.path.exists(l_file_path):
            print(f"No matching L file for F file: {file_path}")
            continue

        data_L = pd.read_excel(l_file_path)

        # Calculate delay and mean
        file_path = os.path.join(root, filename)
        data_F = pd.read_excel(file_path)

        # Extract the values
        
        a = (data_F['Amplitude, AU'].head(10)/data_F['Baseline, AU'].head(10)).values.reshape(-1,1)
        b = (data_L['Amplitude, um'].head(10)/data_L['Baseline, um'].head(10)).values.reshape(-1,1)
        min_length = min(len(a), len(b))
        a = a[:len(b)]
        a = a[:min_length]
        

        # Create a Linear Regression object
        regressor = LinearRegression()

        # Fit the model to the data (learn the coefficients)
        regressor.fit(b, a)

        # Predict the values
        a_pred = regressor.predict(b)

        # Generate and save the figure
        plt.figure(figsize=(10, 8))
        plt.scatter(b, a)
        sns.regplot(x=b.ravel(), y=a.ravel(), ci=95)
        plt.plot(b, a_pred, color='red')
        
        # Save the figure as a PNG file in the output folder
        plt.xlabel('dLL',fontsize=24, weight='bold')
        plt.ylabel('dFF',fontsize=24, weight='bold')
        plt.tick_params(axis = "both" , labelsize = 18)
        plt.tight_layout()
        output_filename = '{}_{}_{}.png'.format(keyword, animal_folder, filename.replace('_F_analysis.xlsx', '_dFFdLL'), dpi = 300)

        
        plt.savefig(os.path.join(root, output_filename))
        plt.close()


D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell1
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell10
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell11
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell12
No matching L file for F file: D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell12\Cell12ProtocolCaf_F_analysis.xlsx
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell13
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell14
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell15
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell16
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell17
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell18
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell19
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell2
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell20
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell21
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell22
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell23
D:

In [2]:
#DELAY FIGURES EXAMPLE
import os
import pandas as pd
import matplotlib.pyplot as plt

# Set the path to the directory containing the files
directory_path = r'D:\My dissertation\obesity\23-08-21\Animal1.mdb'

# Loop over the directories, subdirectories, and files in the directory path
for root, dirs, files in os.walk(directory_path):
    # Loop over the files in the directory
    for filename in files:
                    
        # Check if the file is a .xlsx file containing the substring "_F_analysis.xlsx"
        if not filename.endswith('.xlsx') or '_F_analysis.xlsx' not in filename:
            continue
        animal_folder = os.path.basename(os.path.dirname(root))
        # Print the current directory path
        print(root)
        # Find the keyword (Running or Sedentary) in the root path
        root_parts = os.path.normpath(root).split(os.sep)
        keyword = ""
        for part in root_parts:
            if "Running" in part or "Sedentary" in part:
                keyword = part
                break
        
        # Extract the folder name from the root path
        folder_name = os.path.basename(root)
       
        # Read in the data
        file_path = os.path.join(root, filename)
        
        # Try to get the matching L file for the current F file
        l_file_path = file_path.replace('_F_analysis.xlsx', '_L_analysis.xlsx')
        if not os.path.exists(l_file_path):
            print(f"No matching L file for F file: {file_path}")
            continue
        
        # Now we process these files
        data_F = pd.read_excel(file_path)
        data_L = pd.read_excel(l_file_path)
        t0_F = data_F['t0, ms'].head(10)
        t0_L = data_L['t0, ms'].head(10)
        delay = t0_L - t0_F
        
        mean_delay = delay.mean()
        
        plt.plot(delay)
        plt.title("Delay", fontsize=20)
        plt.tight_layout()
        plt.axhline(y=mean_delay, color='black', linestyle='--')
        plt.tick_params(axis = "both" , labelsize = 18)
        plt.ylabel('Time, ms', fontsize=24, weight='bold')
        plt.xlabel("N transient", fontsize=24, weight='bold')
        plt.tick_params(axis = "both" , labelsize = 18)
        plt.tight_layout()           

        output_filename = '{}_{}_{}.png'.format(keyword, animal_folder, filename.replace('_F_analysis.xlsx', '_delay'))

        
        plt.savefig(os.path.join(root, output_filename), dpi = 300)
        plt.close()



D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell1
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell10
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell11
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell12
No matching L file for F file: D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell12\Cell12ProtocolCaf_F_analysis.xlsx
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell13
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell14
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell15
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell16
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell17
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell18
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell19
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell2
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell20
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell21
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell22
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell23
D:

In [3]:
#A/B linear regression EXAMPLE
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns

# Set the path to the directory containing the files
directory_path = r'D:\My dissertation\obesity'

# Loop over the directories, subdirectories, and files in the directory path
for root, dirs, files in os.walk(directory_path):
    # Loop over the files in the directory
    for filename in files:
                    
        # Check if the file is a .xlsx file containing the substring "_F_analysis.xlsx"
        if not filename.endswith('.xlsx') or '_F_analysis.xlsx' not in filename:
            continue
        animal_folder = os.path.basename(os.path.dirname(root))
        # Print the current directory path
        print(root)
        # Find the keyword (Running or Sedentary) in the root path
        root_parts = os.path.normpath(root).split(os.sep)
        keyword = ""
        for part in root_parts:
            if "Running" in part or "Sedentary" in part:
                keyword = part
                break
        
        # Extract the folder name from the root path
        folder_name = os.path.basename(root)
       
        # Read in the data
        file_path = os.path.join(root, filename)
        


        

        # Calculate delay and mean

        data_F = pd.read_excel(file_path)

        # Extract the values
        b = data_F['Baseline, AU'].head(10)
        b = b.values.reshape(-1,1)# values converts it into a numpy array
        a = data_F['Amplitude, AU'].head(10)
        a = a.values.reshape(-1,1)
        

        # Create a Linear Regression object
        regressor = LinearRegression()

        # Fit the model to the data (learn the coefficients)
        regressor.fit(b, a)

        # Predict the values
        a_pred = regressor.predict(b)

        # Generate and save the figure
        plt.figure(figsize=(10, 8))
        plt.tight_layout()
        plt.tick_params(axis = "both" , labelsize = 18)

        plt.scatter(b, a)
        sns.regplot(x=b.ravel(), y=a.ravel(), ci=95)
        plt.plot(b, a_pred, color='red')
        
        # Save the figure as a PNG file in the output folder
        plt.tick_params(axis = "both" , labelsize = 18)
        plt.xlabel('Baseline, AU',fontsize=24, weight='bold')
        plt.ylabel('Amplitude, AU',fontsize=24, weight='bold')

        plt.tight_layout()           

        output_filename = '{}_{}_{}.png'.format(keyword, animal_folder, filename.replace('_F_analysis.xlsx', '_ab'))

        
        plt.savefig(os.path.join(root, output_filename), dpi = 300)
        plt.close()

D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell1
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell11
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell12
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell13
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell14
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell17
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell2
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell20
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell21
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell22
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell25
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell28
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell3
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell4
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell5
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell6
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell7
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell8


In [13]:
#Transients_F
import os
import pandas as pd
import matplotlib.pyplot as plt

# Directory where your Excel files are located
directory_path = r'D:\My dissertation\obesity'

# Loop over the directories, subdirectories, and files in the directory path
for root, dirs, files in os.walk(directory_path):
    # Loop over the files in the directory
    for filename in files:
                    
        # Check if the file is a .xlsx file containing the substring "_F_analysis.xlsx"
        if not filename.endswith('.xlsx') or '_F_analysis.xlsx' not in filename:
            continue
        animal_folder = os.path.basename(os.path.dirname(root))
        # Print the current directory path
        print(root)
# Find the keyword (Running or Sedentary) in the root path
        root_parts = os.path.normpath(root).split(os.sep)
        keyword = ""
        for part in root_parts:
            if "Running" in part or "Sedentary" in part:
                keyword = part
                break
        
       # Extract the folder name from the root path
        folder_name = os.path.basename(root)

       
        # Read in the data
        file_path = os.path.join(root, filename)
        data_ca = pd.read_excel(file_path, sheet_name=1)
        data_amplitude = pd.read_excel(file_path)

        row_amplitude = data_amplitude.head(10)
        if not row_amplitude.empty:
            t0 = row_amplitude["t0, ms"].iloc[-1]
            cut = row_amplitude.index[-1] + 1
        else:
               # Handle the case when row_amplitude is empty.
            # You can set a default value for t0 or skip the current file.
            print(f"Skipping {filename} due to missing data.")
            continue


        filtered_data_traces = data_ca[data_ca['time, ms'] < 11000]

        Signal = filtered_data_traces['F, AU']
        Aproximation = filtered_data_traces['Approximated F, AU']
        time = filtered_data_traces['time, ms']

        res = Signal - Aproximation
        sigma = res.std()
        Amplitude = data_amplitude['Amplitude, AU']

        
        t0s = data_amplitude['t0, ms'].iloc[:10]
        baseline = data_amplitude['Baseline, AU'].iloc[:10]
        A = Amplitude.mean()
        SNR = A / (3 * sigma)

       
            

        # Generate the figure
        fig, (ax1, ax2) = plt.subplots(2, gridspec_kw={'height_ratios': [5, 3]}, sharex=True,figsize=(12, 8))
        ax1.scatter(time, Signal, color='grey', s = 3)
        ax1.plot(time, Aproximation, color='red')
        ax1.set_ylabel('Amplitude (AU)', fontsize=14)
        ax1.plot(t0s, baseline, marker=6, linestyle='none', color='black')
        ax1.set_ylim([0, 500])  # Fix the y-axis from 0 to 500

        ax2.plot(time, res, color='black')
        ax2.set_xlabel('Time (s)', fontsize=14)
        ax2.set_ylabel('res', fontsize=14)
        ax2.set_ylim([0, 500])  # Fix the y-axis from 0 to 500

        ax1.annotate(f'{filename}_{animal_folder}', xy=(1, 1), xycoords='axes fraction', fontsize=15,
                     xytext=(-5, -5), textcoords='offset points',
                     ha='right', va='top', color = 'black')

        ax1.set_xlim([0, 11000])
        ax1.set_ylim([0, 300])  # Fix the y-axis from 0 to 500
        ax2.set_xlim([0, 11000])
        ax2.set_ylim([-25, 25])
        
        fig.text(0.50, 0.95, f'SNR: {SNR:.2f}', fontsize=15,
         color='black', weight='bold', ha='left', va='top')
        
        plt.subplots_adjust(hspace=0.1)
        plt.tight_layout()
        
        output_filename = '{}_{}_{}.png'.format(keyword, animal_folder, filename.replace('_F_analysis.xlsx', '_F_analysis'))

        
        plt.savefig(os.path.join(root, output_filename), dpi = 300)
        plt.close()


D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell1
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell11
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell12
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell13
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell14
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell17
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell2
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell20
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell21
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell22
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell25
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell28
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell3
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell4
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell5
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell6
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell7
D:\My dissertation\obesity\23-03-20\23-03-20.mdb\Cell8


In [1]:
#Transients_L
import os
import pandas as pd
import matplotlib.pyplot as plt

# Directory where your Excel files are located
directory_path = r'D:\My dissertation\obesity\23-08-21\Animal1.mdb'

# Iterate over all files in the directory
for root, dirs, files in os.walk(directory_path):
    # Loop over the files in the directory
    for filename in files:
        # Check if the file is a .xlsx file containing the substring "_L_analysis.xlsx"
        if not filename.endswith('_L_analysis.xlsx'):
            continue

        # Get simple "_L.xlsx" filename and check if it exists
        l_filename = filename.replace("_L_analysis.xlsx", "_L.xlsx")
        
        if filename.endswith('_L.xlsx'):

            # Get corresponding "_L_analysis.xlsx" filename and check if it exists
            l_analysis_filename = filename.replace("_L.xlsx", "_L_analysis.xlsx")
            if l_analysis_filename not in files:
                print(f"Missing corresponding _L_analysis.xlsx file for {filename}. Continuing with _L.xlsx operations...")

                animal_folder = os.path.basename(os.path.dirname(root))
                print(root)
                root_parts = os.path.normpath(root).split(os.sep)
                keyword = ""
                for part in root_parts:
                    if "Running" in part or "Sedentary" in part:
                        keyword = part
                        break

                l_file_path = os.path.join(root, filename)
                data = pd.read_excel(l_file_path, sheet_name= "Transients")
                data = data[data['time, ms'] < 11000]
                Signal = data['L, um']
                time = data['time, ms']

                # You won't have Aproximation here as it comes from the "_L_analysis.xlsx" file
                # Similarly, 'res' calculation will be affected as it uses Aproximation
                # Make necessary adjustments

                fig, ax1 = plt.subplots(figsize=(12, 8))  # Only one plot is needed
                ax1.scatter(time, Signal, color='grey', s=3)
                ax1.set_ylabel('L, um', fontsize=14)
                ax1.set_xlabel('Time (s)', fontsize=14)
                ax1.set_xlim([0, 11000])
                ax1.set_ylim([1, 2]) 

                output_filename = '{}_{}_{}.png'.format(keyword, animal_folder, filename.replace('.xlsx', ''))
                plt.savefig(os.path.join(root, output_filename), dpi=300)
                plt.close()        
        
        elif l_filename not in files:
            print(f"Missing corresponding _L.xlsx file for {filename}.")
            continue

        animal_folder = os.path.basename(os.path.dirname(root))

        # Print the current directory path
        print(root)

        # Find the keyword (Running or Sedentary) in the root path
        root_parts = os.path.normpath(root).split(os.sep)
        keyword = ""
        for part in root_parts:
            if "Running" in part or "Sedentary" in part:
                keyword = part
                break

        # Read in the data
        file_path = os.path.join(root, filename)
        l_file_path = os.path.join(root, l_filename)

        data_ap = pd.read_excel(file_path, sheet_name="Transients")
        data_amplitude = pd.read_excel(file_path, sheet_name = "Parameters")
        try:
            data = pd.read_excel(l_file_path, sheet_name="Transients")
        except:
            data = pd.read_excel(l_file_path, sheet_name="Length")

        # Filter values less than 11000
        data_ap = data_ap[data_ap['time, ms'] < 11000]
        data = data_ap[data['time, ms'] < 11000]
      

        Aproximation = data_ap['Approximated L, um']
        time_ap = data_ap['time, ms']
        Signal = data['L, um']
        time = data['time, ms']
        
        res = Signal - Aproximation
        sigma = res.std()
        Amplitude = data_amplitude['Amplitude, um'].iloc[:10]

        
        t0s = data_amplitude['t0, ms'].iloc[:10]
        baseline = data_amplitude['Baseline, um'].iloc[:10]
        A = Amplitude.mean()
        SNR = - A / (3 * sigma)

        

        # Calculations for res, sigma, Amplitude, and SNR can be placed here

        # Generate the figure
        fig, (ax1, ax2) = plt.subplots(2, gridspec_kw={'height_ratios': [5, 3]}, sharex=True,figsize=(12, 8))
        ax1.scatter(time, Signal, color='grey', s=3)
        ax1.plot(time_ap, Aproximation, color='red')
        ax1.set_ylabel('L, um', fontsize=14)
        ax1.set_xlabel('Time (s)', fontsize=14)
        ax1.set_xlim([0, 11000])
        ax1.set_ylim([1, 2.2]) 
        ax1.annotate(f'{filename}_{animal_folder}', xy=(1, 1), xycoords='axes fraction', fontsize=15,
                     xytext=(-5, -5), textcoords='offset points',
                     ha='right', va='top', color = 'black')

        
        ax2.plot(time, res, color='black')
        ax2.set_xlabel('Time (s)', fontsize=14)
        ax2.set_ylabel('res', fontsize=14)
        ax2.set_ylim([-0.05, 0.05]) 
        
        fig.text(0.50, 0.95, f'SNR: {SNR:.2f}', fontsize=15,
         color='black', weight='bold', ha='left', va='top') #SNR text
        
        plt.subplots_adjust(hspace=0.1)
        plt.tight_layout()

        # Annotations and other figure adjustments can be placed here

        output_filename = '{}_{}_{}.png'.format(keyword, animal_folder, filename.replace('_L_analysis.xlsx', '_L_analysis'))
        plt.savefig(os.path.join(root, output_filename), dpi=300)
        plt.close()


D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell1


C:\Users\julba\AppData\Local\Temp\ipykernel_25492\4186229681.py:87: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data_ap[data['time, ms'] < 11000]


D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell10


C:\Users\julba\AppData\Local\Temp\ipykernel_25492\4186229681.py:87: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data_ap[data['time, ms'] < 11000]


D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell11


...

C:\Users\julba\AppData\Local\Temp\ipykernel_25492\4186229681.py:87: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data_ap[data['time, ms'] < 11000]


D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell8


C:\Users\julba\AppData\Local\Temp\ipykernel_25492\4186229681.py:87: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data_ap[data['time, ms'] < 11000]


D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell9


C:\Users\julba\AppData\Local\Temp\ipykernel_25492\4186229681.py:87: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data_ap[data['time, ms'] < 11000]
